<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Creado en 2017-2 por Equipo Docente IIC2233, 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados.</font>
</p>

# ¿Cómo iterar sobre mis propias estructuras de datos? 

Hasta ahora hemos visto como implementar varios métodos de árboles, listas ligadas y grafos. Sin embargo, no hemos visto cómo recorrer estas estructuras utilizando `for`, como lo haríamos con una lista. Para esto hay que entender dos conceptos claves: iterable e iterador.

Un **iterable** es cualquier objeto sobre el cual se puede iterar. Por lo mismo, cualquier iterable podría aparecer al lado derecho de un _for loop_ (`for i in iterable:`). Un iterable contiene el método **`__iter__()`**. Se puede iterar todas las veces que uno quiera sobre un iterable, como en el caso de las listas por ejemplo. No es necesario que este objeto se pueda indexar. Por ejemplo los `set`s no se indexan, pero si podemos iterar sobre ellos.

In [1]:
conjunto = {1, 3, 4, 6}

for i in conjunto:
    print(i)

1
3
4
6


Un **iterador** es un objeto que itera sobre un iterable, y es el objeto retornado por el método `__iter__()`. Además, contiene el método `__next__()` que nos retorna el siguiente elemento (uno a la vez).

Recuerden 

> Un iterable debe tener el método `__iter__` implementado, y debe retornar **siempre** un iterador. Por su parte, un iterador es un objeto que tiene el método `__next__` implementado, es decir puedo hacer `next(objeto)` y esto retornará un **valor**.

Hay (al menos) dos formas de implementar estos métodos en sus clases. Por ahora solo les enseñaremos una. Cuando vemamos Programación Funcional les mostraremos la otra forma.

### Datos
Primero generaremos datos con los que vamos a trabajar.

In [2]:
class Nodo:
    
    def __init__(self, valor, siguiente):
        self.valor = valor
        self.siguiente = siguiente
    
    def __repr__(self):
        return "{} - {}".format(self.valor, self.siguiente if self.siguiente else "end")


datos = Nodo(1, Nodo(2, Nodo(3, Nodo(4, Nodo(5, None)))))
print(datos)

1 - 2 - 3 - 4 - 5 - end


Primero debemos crear la clase `Iterable` que implementará el método `__iter__`. 

In [3]:
class Iterable:
    
    def __init__(self, objeto):
        self.objeto = objeto
    
    def __iter__(self):
        return Iterador(self.objeto)

Ahora creamos la clase `Iterador` que será la encargada de iterar sobre el iterable. Ésta debe implementar el método `__next__` que retornará los valores.

El método `__iter__` sólo debe retornar `self`. Es posible no implementarlo. En ese caso, deberán primero obtener el iterador del iterable (`iter(iterable)`) y luego podrán usar ese iterador en sus ciclos `for` y `while`. 

El iterador no se puede reiniciar.

In [4]:
class Iterador:
    
    def __init__(self, iterable):
        self.iterable = iterable
    
    def __iter__(self): 
        return self
    
    def __next__(self):
        if self.iterable is None:
            raise StopIteration("Llegamos al final")
        else:           
            to_return = self.iterable
            self.iterable = self.iterable.siguiente
            return to_return    

In [5]:
iterable = Iterable(datos)
for i in iterable:
    print(i)

1 - 2 - 3 - 4 - 5 - end
2 - 3 - 4 - 5 - end
3 - 4 - 5 - end
4 - 5 - end
5 - end


Esto sería obligatorio si es que no implementan el método `__iter__` en el iterable:

In [6]:
iterable = Iterable(datos)
iterator = iter(iterable) # Obtener el iterador
for i in iterator: # Iterar con el iterador
    print(i)

1 - 2 - 3 - 4 - 5 - end
2 - 3 - 4 - 5 - end
3 - 4 - 5 - end
4 - 5 - end
5 - end


Si paramos la iteración:

In [7]:
iterable = Iterable(datos)
iterator = iter(iterable)
for i in iterator:
    print(i)
    if i.valor >= 3:
        break

1 - 2 - 3 - 4 - 5 - end
2 - 3 - 4 - 5 - end
3 - 4 - 5 - end


Podemos volver a empezar **obteniendo otro iterador**.

In [8]:
for i in iterable:
    print(i)

1 - 2 - 3 - 4 - 5 - end
2 - 3 - 4 - 5 - end
3 - 4 - 5 - end
4 - 5 - end
5 - end


Cada iterador tiene su propia memoria que no depende del iterable

In [9]:
for i in iterator:
    print(i)

4 - 5 - end
5 - end


In [10]:
for i in iterator: # Ya usamos este iterador para recorrer el iterable. No lo podemos utilizar de nuevo.
    print(i)

## Generadores

Los generadores son un caso especial de los **iteradores** (en el material de la semana pasada). Los generadores nos permiten iterar sobre secuencias de datos sin la necesidad de almacenarlos en alguna estructura de datos, evitando el uso innecesario de memoria. 

Una vez que terminamos de iterar sobre un generador, el generador desaparece. Esto es muy útil cuando queremos realizar cálculos sobre secuencias de números que sólo nos sirven para ese cálculo en particular. La sintaxis para crear generadores es muy parecida a la comprensión de listas, sólo que en vez de paréntesis cuadrados [] usamos paréntesis normales ():

In [19]:
from sys import getsizeof

In [20]:
generador_pares = (2*i for i in range(10))
#por el sólo hecho de usar paréntesis 
#significa que estamos creando un generador
print('tamaño del generador:', getsizeof(generador_pares))

lista_pares = [2*i for i in range(10)]#c usa más memoria que a

print('tamaño de la lista:', getsizeof(lista_pares))

tamaño del generador: 88
tamaño de la lista: 192


A continuación podemos observar que una vez que ya hemos iterado sobre el generador, no lo podemos volver a utilizar.

In [21]:
print('valores del generador')
for i in generador_pares:
    print(i)

print('valores del generador')
for i in generador_pares:
    print(i)
#como ya terminamos de iterar sobre a
#la secuencia desaparece

valores del generador
0
2
4
6
8
10
12
14
16
18
valores del generador


## Funciones Generadoras

Las funciones en Python también tienen la posibilidad de funcionar como generadores, a través de `yield`. El statement `yield` reemplaza a `return`, por un lado se encarga de retornar el valor pero además nos asegura que en la próxima llamada a la función, ésta será ejecutada partiendo desde el punto donde quedó en la ejecución anterior. En otras palabras, trabajamos con una función que una vez que "retorna" un valor a través de `yield`, está transfierendo el control sólo en forma temporal, asumiendo que pronto será utilizada nuevamente para "generar" más valores. Al llamar a una función generadora se crea un objeto generador, sin embargo, esto no comienza a ejecutar la función. Ejemplo:

In [11]:
def conteo_dec(n):
    print("Contando en forma decreciente desde {}".format(n))
    while n > 0:
        yield n
        n -= 1

La función se ejecuta una vez que llamamos a `next` del objeto generado por la función, que retorna un generador.

In [13]:
x = conteo_dec(10)#notar que aquí no se imprime nada
print("{}\n".format(x))#aquí sólo se imprime el objeto
y = conteo_dec(5)
print(next(y))
print(next(y))
print(next(y))
print(next(y))
#for i in conteo_dec(5):
#    print(i)

<generator object conteo_dec at 0x10ea44d58>

Contando en forma decreciente desde 5
5
4
3
2


In [14]:
def fibonacci():
    a,b = 0,1
    while True:
        yield b
        a, b = b, a + b

f = fibonacci()
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
print(next(f))
g1 = [next(f) for i in range(10)]
print(g1)
g2 = (next(f) for i in range(10))
for a in g2: print(a)

1
1
2
3
5
8
[13, 21, 34, 55, 89, 144, 233, 377, 610, 987]
1597
2584
4181
6765
10946
17711
28657
46368
75025
121393


In [15]:
import numpy as np
def maximo(values):
    temp_maxim = -np.infty
    for v in values:
        if v > temp_maxim:
            temp_maxim = v
        yield temp_maxim
        
elements = [10, 14, 7, 9, 12, 19, 33]
res = maximo(elements)
print(next(res))
print(next(res))
print(next(res))
print(next(res))
print(next(res))
print(next(res))
print(next(res))#aquí se acabó la lista

10
14
14
14
14
19
33


### También podemos interactuar con la función enviando mensajes 

El método `send()` permite enviar un valor hacia el generador, lo que significa que la expresión `yield` lo recibirá. El valor enviado puede ser usado para asignarlo a otra variable, por ejemplo: `v = yield self.value`

In [16]:
def mov_avg():
    print("entrando...")
    total = float((yield))
    cont = 1
    print("total = {}".format(total))
    while True:
        print("loop del while...")
        i = yield total / cont #aquí el i recibe el mensaje, además se está retornando total/count
        cont += 1
        total += i
        print("i = {}".format(i))
        print("total = {}".format(total))
        print("cont = {}".format(cont))

Notar que el código debe ejecutar al menos hasta el primer `yield` para poder empezar a aceptar valores desde `send()`. Así siempre es necesario llamar a `next()` (o `send(None)`) una vez después de haber creado el generador para poder comenzar a enviarle datos.

In [17]:
m = mov_avg()
print("entrando al primer next")
next(m)#avanzamos al primer yield
print("saliendo del primer next")
m.send(10)
print("entrando al send de 5")
m.send(5)
print("entrando al send de 0")
m.send(0)
print("entrando al otro send de 0")
m.send(0)
print("entrando al send de 20")
m.send(20)

entrando al primer next
entrando...
saliendo del primer next
total = 10.0
loop del while...
entrando al send de 5
i = 5
total = 15.0
cont = 2
loop del while...
entrando al send de 0
i = 0
total = 15.0
cont = 3
loop del while...
entrando al otro send de 0
i = 0
total = 15.0
cont = 4
loop del while...
entrando al send de 20
i = 20
total = 35.0
cont = 5
loop del while...


7.0

In [18]:
m = mov_avg()
print(next(m))
print(m.send(10))
print(m.send(5))
print(m.send(0))
print(m.send(0))
print(m.send(20))

entrando...
None
total = 10.0
loop del while...
10.0
i = 5
total = 15.0
cont = 2
loop del while...
7.5
i = 0
total = 15.0
cont = 3
loop del while...
5.0
i = 0
total = 15.0
cont = 4
loop del while...
3.75
i = 20
total = 35.0
cont = 5
loop del while...
7.0
